<h1>
    <center>
        <span style="font-family:Arial">
            Imagerie Numérique
        </span>
    </center>
</h1>
<h2>
    <center>
        <span style="font-family:Arial">
            Image Fusion with Guided Filtering
        </span>
    </center>
</h2>
MVA 2021,
Gabriel Belouze & Raphaël Rozenberg

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio as iio
from pathlib import Path

# Our own implementation
import gf.filters as filters
import gf.data as data
import gf.fusion as fusion

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 160
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

## Table of content
0. **[Preliminaries](#preliminaries)**
1. **[Guided filtering](#guided-filtering)**
    1. [Gray guide](#gf:gray-guide)
    2. [RGB guide](#gf:rgb-guide)
2. **[Image Fusion with guided filtering](#image-fusion)**
    1. [Step by step](#fusion:step-by-step)
    2. [Examples](#fusion:examples)
3. **[Experiments](#experiments)**

# Preliminaries <a class="anchor" id="preliminaries"></a>

## Load data

In [ ]:
multi_exposure_dataset = data.MultiviewDataset(Path("../data/MEFDatabase/source/"))
multi_focus_dataset = data.MultiviewDataset(Path("../data/lytro"))

multi_exposure_sample = multi_exposure_dataset[0]
multi_focus_sample = multi_focus_dataset[0]

## Show data

In [ ]:
def gray_to_rgb(im):
    return np.stack([im, im, im], axis=-1)

def plot_images(*ims):
    plt.axis('off')

    max_ndim = max([im.ndim for im in ims])
    if max_ndim == 3:
        ims = [im if im.ndim == 3 else gray_to_rgb(im) for im in ims]
        
    im = np.hstack(ims)
    if im.ndim == 3:
        plt.imshow(im)
    else:
        plt.imshow(im, cmap='gray')

In [ ]:
plot_images(*multi_exposure_sample)

In [ ]:
plot_images(*multi_focus_sample)

# Guided filtering <a class="anchor" id="guided-filtering"></a>

![](images/guided_filter_schematic.png)

## Gray guide <a class="anchor" id="gf:gray-guide"></a>

In [ ]:
input = multi_focus_sample[0].mean(axis=-1)
guide = multi_focus_sample[1].mean(axis=-1)
output = filters.guided_filter(input, guide, r=20, eps=5e-2)

In [ ]:
plot_images(input, guide, output)

Notably, the edges of the buildings in the backgrounds remain sharp.

## RGB guide <a class="anchor" id="gf:rgb-guide"></a>

In [ ]:
input = multi_focus_sample[0]
guide = multi_focus_sample[1]
output = filters.guided_filter(input, guide, r=20, eps=5e-2)

In [ ]:
plot_images(input, guide, output)

# Image fusion with guided filtering <a class="anchor" id="image-fusion"></a>

![](images/image_fusion_schematic.png)

## Step by step <a class="anchor" id="fusion:step-by-step"></a>

#### Base / Detail decomposition
First images are split into a base layer and a detail layer. Each layer is then treated independantly, and fused back in only at the very end.

In [ ]:
# TODO illustrate with code

#### Weight map
Weight maps are constructed to be $1$ at pixel $i$ if the image has the highest saliency (i.e. gradient norm) at pixel $i$, and $0$ otherwise.

In [ ]:
# TODO illustrate with code

#### Refined weight map
The key idea is to use guided filtering with the original image as guidance. This is in particular useful to edge-align weight maps.

In [ ]:
# TODO illustrate with code

#### Fusion
Those refined weight maps are used to add images in each layers. Finally, the layers are added up to produce a single final image.

In [ ]:
# TODO illustrate with code

## Examples <a class="anchor" id="fusion:examples"></a>

In [ ]:
exposure_gff = fusion.gff(multi_exposure_sample)
multi_exposure_fused = exposure_gff.fusion()

focus_gff = fusion.gff(multi_focus_sample)
multi_focus_fused = focus_gff.fusion()

In [ ]:
plot_images(*multi_exposure_sample, multi_exposure_fused)
plt.show()

plot_images(*multi_focus_sample, multi_focus_fused)
plt.show()

Conclusion : it is extremely cool

# Experiments <a class="anchor" id="experiments"></a>

## Averaging the coefficients $a$ and $b$

In guided filtering, the final value for $O_i$ is averaged over all windows where $O_i$ was computed. This amounts to averaging the coefficients :
$$O_i = \bar{a}_i O_i + \bar{b}_i$$
What happens if we skip the averaging step and instead choose $O_i = a_i I_i + b_i$ ?

In [ ]:
input = multi_focus_sample[0].mean(axis=-1)
guide = multi_focus_sample[1].mean(axis=-1)
output = filters.guided_filter(input, guide, r=20, eps=5e-2, average_in_window=False)

In [ ]:
plot_images(input, guide, output)

There is a shadowing effect (look around the hat and the background buildings). TODO why ?